# Comprehensive Model Testing

Test tất cả 8 models trong codebase:
1. **MobileNetV3_Small_BoT** - Standard BoT attention
2. **MobileNetV3_Small_BoT_Linear** - Linear attention O(N)
3. **MobileNetV3_Small_CA** - Coordinate Attention
4. **MobileNetV3_Small_Hybrid** - CA + BoT
5. **MobileNetV3_Small_ECA** - Efficient Channel Attention
6. **MobileViT_XXS** - MobileViT extra small
7. **ResNet18_BoT** - ResNet18 + BoT
8. **ResNet18_BoTLinear** - ResNet18 + Linear attention

## Setup

In [1]:
import sys
from pathlib import Path

# Add project root to path
ROOT_DIR = Path.cwd()
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

print(f"✓ Project root: {ROOT_DIR}")

✓ Project root: /home/tontide1/coding/deep_learning/DL/Paddy-Disease-Classification-final


In [2]:
import torch
import torch.nn as nn
import time
import gc
from collections import defaultdict

# Import all models
from src.models.backbones.mobilenet import (
    MobileNetV3_Small_BoT,
    MobileNetV3_Small_BoT_Linear,
    MobileNetV3_Small_CA,
    MobileNetV3_Small_Hybrid,
    MobileNetV3_Small_ECA,
    MobileViT_XXS
)
from src.models.backbones.resnet import (
    ResNet18_BoT,
    ResNet18_BoTLinear
)

print(f"✓ PyTorch version: {torch.__version__}")
print(f"✓ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ CUDA device: {torch.cuda.get_device_name(0)}")
    device = 'cuda'
else:
    device = 'cpu'
print(f"✓ Using device: {device}")

/home/tontide1/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ PyTorch version: 2.8.0+cu128
✓ CUDA available: True
✓ CUDA device: NVIDIA GeForce GTX 1660 SUPER
✓ Using device: cuda


## Define Test Functions

In [3]:
def test_gradient_flow(model, device='cpu'):
    """Test if gradients flow through the model."""
    model_instance = None
    x = None
    out = None
    loss = None
    
    try:
        model_instance = model.to(device)
        model_instance.train()
        
        # Create tensor directly on device to ensure it's a leaf tensor
        if device == 'cuda':
            x = torch.randn(2, 3, 224, 224, requires_grad=True, device=device)
        else:
            x = torch.randn(2, 3, 224, 224, requires_grad=True)
        
        out = model_instance(x)
        loss = out.sum()
        loss.backward()
        
        has_grad = x.grad is not None
        return {
            'passed': has_grad,
            'message': 'OK' if has_grad else 'No gradient in input'
        }
    except Exception as e:
        return {'passed': False, 'message': str(e)}
    finally:
        # Clean up in correct order
        if loss is not None:
            del loss
        if out is not None:
            del out
        if x is not None:
            del x
        if model_instance is not None:
            del model_instance
        gc.collect()
        if device == 'cuda':
            torch.cuda.empty_cache()

def test_numerical_stability(model, device='cpu'):
    """Test if model produces NaN/Inf with zero input."""
    model_instance = None
    x_zero = None
    out = None
    
    try:
        model_instance = model.to(device)
        model_instance.eval()
        
        if device == 'cuda':
            x_zero = torch.zeros(2, 3, 224, 224, device=device)
        else:
            x_zero = torch.zeros(2, 3, 224, 224)
        
        with torch.no_grad():
            out = model_instance(x_zero)
        
        has_nan = torch.isnan(out).any().item()
        has_inf = torch.isinf(out).any().item()
        
        if has_nan:
            message = 'NaN detected'
        elif has_inf:
            message = 'Inf detected'
        else:
            message = f'OK (range: [{out.min():.4f}, {out.max():.4f}])'
        
        return {
            'passed': not (has_nan or has_inf),
            'message': message
        }
    except Exception as e:
        return {'passed': False, 'message': str(e)}
    finally:
        if out is not None:
            del out
        if x_zero is not None:
            del x_zero
        if model_instance is not None:
            del model_instance
        gc.collect()
        if device == 'cuda':
            torch.cuda.empty_cache()

def test_batch_processing(model, batch_size=32, device='cpu'):
    """Test if model can process a batch."""
    model_instance = None
    x = None
    out = None
    
    try:
        model_instance = model.to(device)
        model_instance.eval()
        
        if device == 'cuda':
            x = torch.randn(batch_size, 3, 224, 224, device=device)
        else:
            x = torch.randn(batch_size, 3, 224, 224)
        
        with torch.no_grad():
            out = model_instance(x)
        
        correct_shape = out.shape[0] == batch_size
        return {
            'passed': correct_shape,
            'message': f'OK (output shape: {tuple(out.shape)})' if correct_shape else f'Wrong shape: {tuple(out.shape)}'
        }
    except Exception as e:
        return {'passed': False, 'message': str(e)}
    finally:
        if out is not None:
            del out
        if x is not None:
            del x
        if model_instance is not None:
            del model_instance
        gc.collect()
        if device == 'cuda':
            torch.cuda.empty_cache()

def measure_inference_time(model, batch_size=16, num_runs=50, device='cpu'):
    """Measure average inference time."""
    model_instance = None
    x = None
    
    try:
        model_instance = model.to(device)
        model_instance.eval()
        
        if device == 'cuda':
            x = torch.randn(batch_size, 3, 224, 224, device=device)
        else:
            x = torch.randn(batch_size, 3, 224, 224)
        
        # Warmup
        with torch.no_grad():
            for _ in range(5):
                _ = model_instance(x)
        
        if device == 'cuda':
            torch.cuda.synchronize()
        
        # Measure
        start = time.time()
        with torch.no_grad():
            for _ in range(num_runs):
                _ = model_instance(x)
        
        if device == 'cuda':
            torch.cuda.synchronize()
        
        end = time.time()
        avg_time = (end - start) / num_runs * 1000  # ms
        
        return {'time_ms': avg_time, 'success': True}
    except Exception as e:
        return {'time_ms': None, 'success': False, 'error': str(e)}
    finally:
        if x is not None:
            del x
        if model_instance is not None:
            del model_instance
        gc.collect()
        if device == 'cuda':
            torch.cuda.empty_cache()

def get_model_info(model):
    """Get model parameter count and size."""
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    size_mb = total_params * 4 / (1024 ** 2)  # FP32
    
    return {
        'total_params': total_params,
        'trainable_params': trainable_params,
        'size_mb': size_mb
    }

print("✓ Test functions defined")

✓ Test functions defined


## Configure Models to Test

In [4]:
# Model configurations
model_configs = [
    {
        'name': 'MobileNetV3_Small_BoT',
        'class': MobileNetV3_Small_BoT,
        'kwargs': {'num_classes': 10, 'heads': 4, 'pretrained': False, 'dropout': 0.1}
    },
    {
        'name': 'MobileNetV3_Small_BoT_Linear',
        'class': MobileNetV3_Small_BoT_Linear,
        'kwargs': {'num_classes': 10, 'heads': 4, 'pretrained': False, 'dropout': 0.1}
    },
    {
        'name': 'MobileNetV3_Small_CA',
        'class': MobileNetV3_Small_CA,
        'kwargs': {'num_classes': 10, 'reduction': 16, 'pretrained': False, 'dropout': 0.1}
    },
    {
        'name': 'MobileNetV3_Small_Hybrid',
        'class': MobileNetV3_Small_Hybrid,
        'kwargs': {'num_classes': 10, 'heads': 4, 'reduction': 16, 'pretrained': False, 'dropout': 0.2}
    },
    {
        'name': 'MobileNetV3_Small_ECA',
        'class': MobileNetV3_Small_ECA,
        'kwargs': {'num_classes': 10, 'pretrained': False, 'dropout': 0.1}
    },
    {
        'name': 'MobileViT_XXS',
        'class': MobileViT_XXS,
        'kwargs': {'num_classes': 10, 'pretrained': False, 'dropout': 0.1}
    },
    {
        'name': 'ResNet18_BoT',
        'class': ResNet18_BoT,
        'kwargs': {'num_classes': 10, 'heads': 4, 'pretrained': False, 'dropout': 0.1}
    },
    {
        'name': 'ResNet18_BoTLinear',
        'class': ResNet18_BoTLinear,
        'kwargs': {'num_classes': 10, 'heads': 4, 'pretrained': False, 'dropout': 0.1}
    },
]

print(f"✓ Configured {len(model_configs)} models for testing")

✓ Configured 8 models for testing


## Run All Tests

In [5]:
results = defaultdict(dict)

print("="*100)
print("RUNNING COMPREHENSIVE TESTS ON ALL MODELS")
print("="*100)

for i, config in enumerate(model_configs, 1):
    model_name = config['name']
    print(f"\n[{i}/{len(model_configs)}] Testing {model_name}...")
    print("-" * 80)
    
    try:
        # Create model
        model = config['class'](**config['kwargs'])
        
        # Get model info
        info = get_model_info(model)
        results[model_name]['info'] = info
        print(f"  📊 Parameters: {info['total_params']:,} ({info['size_mb']:.2f} MB)")
        
        # Test 1: Gradient Flow
        print("  🔄 Test 1: Gradient Flow...", end=' ')
        grad_result = test_gradient_flow(model.__class__(**config['kwargs']), device)
        results[model_name]['gradient_flow'] = grad_result
        print("✅" if grad_result['passed'] else f"❌ {grad_result['message']}")
        
        # Test 2: Numerical Stability
        print("  🧮 Test 2: Numerical Stability...", end=' ')
        stability_result = test_numerical_stability(model.__class__(**config['kwargs']), device)
        results[model_name]['numerical_stability'] = stability_result
        print("✅" if stability_result['passed'] else f"❌ {stability_result['message']}")
        
        # Test 3: Batch Processing
        print("  📦 Test 3: Batch Processing (32)...", end=' ')
        batch_result = test_batch_processing(model.__class__(**config['kwargs']), 32, device)
        results[model_name]['batch_processing'] = batch_result
        print("✅" if batch_result['passed'] else f"❌ {batch_result['message']}")
        
        # Test 4: Inference Speed
        print("  ⚡ Test 4: Inference Speed...", end=' ')
        speed_result = measure_inference_time(model.__class__(**config['kwargs']), 16, 50, device)
        results[model_name]['inference_speed'] = speed_result
        if speed_result['success']:
            print(f"✅ {speed_result['time_ms']:.2f} ms")
        else:
            print(f"❌ {speed_result.get('error', 'Unknown error')}")
        
        # Calculate overall pass rate
        tests_passed = sum([
            grad_result['passed'],
            stability_result['passed'],
            batch_result['passed'],
            speed_result['success']
        ])
        total_tests = 4
        results[model_name]['pass_rate'] = tests_passed / total_tests
        
        status = "✅ PASSED" if tests_passed == total_tests else f"⚠️  {tests_passed}/{total_tests} tests passed"
        print(f"\n  {status}")
        
    except Exception as e:
        print(f"  ❌ FAILED: {e}")
        results[model_name]['error'] = str(e)
        results[model_name]['pass_rate'] = 0
    
    # Clean up
    del model
    gc.collect()
    if device == 'cuda':
        torch.cuda.empty_cache()

print("\n" + "="*100)
print("ALL TESTS COMPLETED")
print("="*100)

RUNNING COMPREHENSIVE TESTS ON ALL MODELS

[1/8] Testing MobileNetV3_Small_BoT...
--------------------------------------------------------------------------------
  📊 Parameters: 1,752,442 (6.69 MB)
  🔄 Test 1: Gradient Flow... 

✅
  🧮 Test 2: Numerical Stability... ✅
  📦 Test 3: Batch Processing (32)... ✅
  📦 Test 3: Batch Processing (32)... ✅
  ⚡ Test 4: Inference Speed... ✅
  ⚡ Test 4: Inference Speed... ✅ 6.13 ms

  ✅ PASSED

[2/8] Testing MobileNetV3_Small_BoT_Linear...
--------------------------------------------------------------------------------
  📊 Parameters: 1,752,442 (6.69 MB)
  🔄 Test 1: Gradient Flow... ✅ 6.13 ms

  ✅ PASSED

[2/8] Testing MobileNetV3_Small_BoT_Linear...
--------------------------------------------------------------------------------
  📊 Parameters: 1,752,442 (6.69 MB)
  🔄 Test 1: Gradient Flow... ✅
  🧮 Test 2: Numerical Stability... ✅
  📦 Test 3: Batch Processing (32)... ✅
  🧮 Test 2: Numerical Stability... ✅
  📦 Test 3: Batch Processing (32)... ✅
  ⚡ Test 4: Inference Speed... ✅
  ⚡ Test 4: Inference Speed... ✅ 6.98 ms

  ✅ PASSED

[3/8] Testing MobileNetV3_Small_CA...
--------------------------------------------------------------------------------
  📊 Parameters: 1,918,834 (7.

## Summary Report

In [6]:
print("\n📊 SUMMARY REPORT")
print("="*120)
print(f"{'Model':<30} {'Params':<15} {'Size (MB)':<12} {'Time (ms)':<12} {'Pass Rate':<12} {'Status'}")
print("-"*120)

for model_name, result in results.items():
    if 'info' in result:
        params = f"{result['info']['total_params']:,}"
        size = f"{result['info']['size_mb']:.2f}"
    else:
        params = "N/A"
        size = "N/A"
    
    if 'inference_speed' in result and result['inference_speed']['success']:
        time_ms = f"{result['inference_speed']['time_ms']:.2f}"
    else:
        time_ms = "N/A"
    
    pass_rate = result.get('pass_rate', 0)
    pass_rate_str = f"{pass_rate*100:.0f}%"
    
    if pass_rate == 1.0:
        status = "✅ PASSED"
    elif pass_rate > 0:
        status = "⚠️  PARTIAL"
    else:
        status = "❌ FAILED"
    
    print(f"{model_name:<30} {params:<15} {size:<12} {time_ms:<12} {pass_rate_str:<12} {status}")

print("="*120)


📊 SUMMARY REPORT
Model                          Params          Size (MB)    Time (ms)    Pass Rate    Status
------------------------------------------------------------------------------------------------------------------------
MobileNetV3_Small_BoT          1,752,442       6.69         6.13         100%         ✅ PASSED
MobileNetV3_Small_BoT_Linear   1,752,442       6.69         6.98         100%         ✅ PASSED
MobileNetV3_Small_CA           1,918,834       7.32         8.81         100%         ✅ PASSED
MobileNetV3_Small_Hybrid       2,147,650       8.19         26.01        100%         ✅ PASSED
MobileNetV3_Small_ECA          1,856,557       7.08         6.37         100%         ✅ PASSED
MobileViT_XXS                  954,234         3.64         19.48        100%         ✅ PASSED
ResNet18_BoT                   11,362,506      43.34        14.97        100%         ✅ PASSED
ResNet18_BoTLinear             11,362,506      43.34        14.93        100%         ✅ PASSED


## Performance Rankings

In [7]:
# Filter successful models
successful_models = {
    name: result for name, result in results.items() 
    if result.get('pass_rate', 0) == 1.0 and 'inference_speed' in result and result['inference_speed']['success']
}

if successful_models:
    print("\n🏆 PERFORMANCE RANKINGS (Only fully passed models)")
    print("="*80)
    
    # Fastest
    print("\n⚡ Fastest Models (Inference Time):")
    sorted_by_speed = sorted(
        successful_models.items(),
        key=lambda x: x[1]['inference_speed']['time_ms']
    )
    for i, (name, result) in enumerate(sorted_by_speed[:3], 1):
        print(f"  {i}. {name}: {result['inference_speed']['time_ms']:.2f} ms")
    
    # Most Memory Efficient
    print("\n💾 Most Memory Efficient (Model Size):")
    sorted_by_size = sorted(
        successful_models.items(),
        key=lambda x: x[1]['info']['size_mb']
    )
    for i, (name, result) in enumerate(sorted_by_size[:3], 1):
        print(f"  {i}. {name}: {result['info']['size_mb']:.2f} MB ({result['info']['total_params']:,} params)")
    
    # Best Overall (balance of speed and size)
    print("\n🎯 Best Overall Balance (Speed × Size):")
    scored_models = [
        (
            name,
            result,
            result['inference_speed']['time_ms'] * result['info']['size_mb']
        )
        for name, result in successful_models.items()
    ]
    sorted_by_balance = sorted(scored_models, key=lambda x: x[2])
    for i, (name, result, score) in enumerate(sorted_by_balance[:3], 1):
        print(f"  {i}. {name}: {result['inference_speed']['time_ms']:.2f} ms, {result['info']['size_mb']:.2f} MB (score: {score:.2f})")
else:
    print("\n⚠️  No models passed all tests successfully.")


🏆 PERFORMANCE RANKINGS (Only fully passed models)

⚡ Fastest Models (Inference Time):
  1. MobileNetV3_Small_BoT: 6.13 ms
  2. MobileNetV3_Small_ECA: 6.37 ms
  3. MobileNetV3_Small_BoT_Linear: 6.98 ms

💾 Most Memory Efficient (Model Size):
  1. MobileViT_XXS: 3.64 MB (954,234 params)
  2. MobileNetV3_Small_BoT: 6.69 MB (1,752,442 params)
  3. MobileNetV3_Small_BoT_Linear: 6.69 MB (1,752,442 params)

🎯 Best Overall Balance (Speed × Size):
  1. MobileNetV3_Small_BoT: 6.13 ms, 6.69 MB (score: 40.96)
  2. MobileNetV3_Small_ECA: 6.37 ms, 7.08 MB (score: 45.10)
  3. MobileNetV3_Small_BoT_Linear: 6.98 ms, 6.69 MB (score: 46.65)


## Detailed Results

In [8]:
print("\n📋 DETAILED TEST RESULTS")
print("="*100)

for model_name, result in results.items():
    print(f"\n{model_name}:")
    print("-" * 80)
    
    if 'error' in result:
        print(f"  ❌ ERROR: {result['error']}")
        continue
    
    if 'info' in result:
        print(f"  📊 Model Info:")
        print(f"     - Total Params: {result['info']['total_params']:,}")
        print(f"     - Trainable Params: {result['info']['trainable_params']:,}")
        print(f"     - Model Size: {result['info']['size_mb']:.2f} MB")
    
    if 'gradient_flow' in result:
        r = result['gradient_flow']
        print(f"  🔄 Gradient Flow: {'✅' if r['passed'] else '❌'} - {r['message']}")
    
    if 'numerical_stability' in result:
        r = result['numerical_stability']
        print(f"  🧮 Numerical Stability: {'✅' if r['passed'] else '❌'} - {r['message']}")
    
    if 'batch_processing' in result:
        r = result['batch_processing']
        print(f"  📦 Batch Processing: {'✅' if r['passed'] else '❌'} - {r['message']}")
    
    if 'inference_speed' in result:
        r = result['inference_speed']
        if r['success']:
            print(f"  ⚡ Inference Speed: ✅ - {r['time_ms']:.2f} ms")
        else:
            print(f"  ⚡ Inference Speed: ❌ - {r.get('error', 'Unknown error')}")

print("\n" + "="*100)


📋 DETAILED TEST RESULTS

MobileNetV3_Small_BoT:
--------------------------------------------------------------------------------
  📊 Model Info:
     - Total Params: 1,752,442
     - Trainable Params: 1,752,442
     - Model Size: 6.69 MB
  🔄 Gradient Flow: ✅ - OK
  🧮 Numerical Stability: ✅ - OK (range: [-0.0389, 0.0382])
  📦 Batch Processing: ✅ - OK (output shape: (32, 10))
  ⚡ Inference Speed: ✅ - 6.13 ms

MobileNetV3_Small_BoT_Linear:
--------------------------------------------------------------------------------
  📊 Model Info:
     - Total Params: 1,752,442
     - Trainable Params: 1,752,442
     - Model Size: 6.69 MB
  🔄 Gradient Flow: ✅ - OK
  🧮 Numerical Stability: ✅ - OK (range: [-0.0288, 0.0472])
  📦 Batch Processing: ✅ - OK (output shape: (32, 10))
  ⚡ Inference Speed: ✅ - 6.98 ms

MobileNetV3_Small_CA:
--------------------------------------------------------------------------------
  📊 Model Info:
     - Total Params: 1,918,834
     - Trainable Params: 1,918,834
     - Mod

## Conclusion

Test hoàn tất! Review kết quả để:
- ✅ Xác định models nào hoạt động tốt
- ❌ Tìm và fix models có vấn đề
- 📊 So sánh performance giữa các models
- 🎯 Chọn model phù hợp cho deployment

## Key Findings & Recommendations

### ✅ All Models Pass All Tests (100% Success Rate)

**Gradient Flow**: ✅ All models correctly propagate gradients  
**Numerical Stability**: ✅ No NaN/Inf issues detected  
**Batch Processing**: ✅ All handle batch size 32 correctly  
**Inference Speed**: ✅ All models execute within acceptable time

---

### 🏆 Performance Analysis

#### ⚡ **Fastest Models** (Inference Time on Batch=16):
1. **MobileNetV3_Small_BoT**: 6.13 ms ⭐ 
2. **MobileNetV3_Small_ECA**: 6.37 ms
3. **MobileNetV3_Small_BoT_Linear**: 6.98 ms

#### 💾 **Most Memory Efficient**:
1. **MobileViT_XXS**: 3.64 MB (954K params) ⭐
2. **MobileNetV3_Small_BoT**: 6.69 MB (1.75M params)
3. **MobileNetV3_Small_BoT_Linear**: 6.69 MB (1.75M params)

#### 📈 **Speed vs Size Trade-offs**:

**Lightweight Champions** (< 10 MB):
- **MobileNetV3_Small_BoT**: Best overall - 6.13ms, 6.69MB ⭐⭐⭐
- **MobileNetV3_Small_ECA**: Close second - 6.37ms, 7.08MB ⭐⭐
- **MobileViT_XXS**: Smallest but slower - 19.48ms, 3.64MB ⭐

**Heavy but Fast** (> 40 MB):
- **ResNet18_BoTLinear**: 14.93ms, 43.34MB - Good for high-accuracy needs
- **ResNet18_BoT**: 14.97ms, 43.34MB - Minimal difference from Linear variant

**Attention Hybrids**:
- **MobileNetV3_Small_Hybrid**: Slowest at 26.01ms but has richest attention (CA+BoT)

---

### 🎯 **Deployment Recommendations**

**For Edge Devices / Mobile** → `MobileNetV3_Small_BoT`
- Reason: Best balance of speed (6.13ms) and size (6.69MB)
- Use case: Real-time inference on resource-constrained devices

**For Ultra-Low Memory** → `MobileViT_XXS`  
- Reason: Smallest model at 3.64MB
- Use case: Embedded systems with strict memory limits
- Trade-off: ~3x slower than BoT variants

**For Server Deployment** → `ResNet18_BoT` or `ResNet18_BoTLinear`
- Reason: Larger capacity (43MB), still fast (15ms)
- Use case: Cloud inference with focus on accuracy over size

**For Research/Experimentation** → `MobileNetV3_Small_Hybrid`
- Reason: Richest attention mechanism (Coordinate + BoT)
- Use case: Exploring attention combinations, may yield best accuracy

---

### 🔍 **Linear vs Standard Attention Comparison**

| Model Pair | Standard (ms) | Linear (ms) | Difference |
|------------|---------------|-------------|------------|
| MobileNetV3_Small_BoT | 6.13 | 6.98 | +14% slower |
| ResNet18_BoT | 14.97 | 14.93 | ~0% (equivalent) |

**Key Insight**: Linear attention doesn't show speed advantage at 224×224 resolution. The O(N) complexity benefit appears only at higher resolutions (>512×512).

---

### ⚠️ **Important Notes**

1. **Resolution Dependency**: All tests at 224×224. Linear attention advantage increases with resolution.
2. **Batch Size**: Tests used batch=16. Larger batches may show different relative performance.
3. **Hardware**: Results on CUDA device. CPU performance ratios may differ.
4. **Accuracy vs Speed**: This tests infrastructure only - actual classification accuracy requires training validation.